# Visualizing and Debugging Graphs

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from tbp.monty.frameworks.utils.logging_utils import load_stats
from tbp.monty.frameworks.utils.plot_utils import plot_graph

In [ ]:
%matplotlib notebook

In [ ]:
# exp data not really loaded, just for the load func to run
# Needs to be an actual path if looking at intermediate models during training
placeholder_exp_path = os.path.expanduser("~/tbp/tbp.monty/projects/monty_runs/logs/feature_pred_tests/")

sensor_pretrained_dict = os.path.expanduser("~/tbp/tbp.monty/projects/monty_runs/pretrained_models_ycb_high_res/supervised_pre_training/pretrained/")
_, _, _, lm_models_sensor = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=sensor_pretrained_dict,
                               )
# feature experiments on mesh models:
mesh_pretrained_dict = os.path.expanduser("~/tbp/tbp.monty/projects/monty_runs/pretrained_models_from_mesh/ycb_mesh_models_low_res/")
mesh_pretrained_dict=os.path.expanduser("~/tbp/tbp.monty/projects/feature_eval_runs/pretrained_models_from_mesh/ycb4_3000p/")
_, _, _, lm_models_mesh = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=mesh_pretrained_dict,
                               )

In [ ]:
pretrain_path = os.path.expanduser("~/tbp/results/monty/pretrained_models/")
pretrained_dict = pretrain_path + "pretrained_ycb_v3/touch_1lm_10distinctobj/pretrained/"

sensor_pretrained_dict = os.path.expanduser("~/tbp/tbp.monty/projects/monty_runs/pretrained_models_ycb_high_res/supervised_pre_training/pretrained/")
_, _, _, lm_models_sensor = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict,
                               )

### Scaling Curvature

In [ ]:
object_n = 'mug'
lm_models = lm_models_sensor

pc_ids_g = lm_models['pretrained'][0][object_n].feature_mapping['gaussian_curvature']
gc = np.array(lm_models['pretrained'][0][object_n].x[:,pc_ids_g[0]:pc_ids_g[1]])
pc_ids_m = lm_models['pretrained'][0][object_n].feature_mapping['mean_curvature']
mc = np.array(lm_models['pretrained'][0][object_n].x[:,pc_ids_m[0]:pc_ids_m[1]])

pc_ids_pc = lm_models['pretrained'][0][object_n].feature_mapping['principal_curvatures']
pc = np.array(lm_models['pretrained'][0][object_n].x[:,pc_ids_pc[0]:pc_ids_pc[1]])

def scale(to_scale):
    negative = to_scale < 0
    scaled = np.sqrt(np.abs(to_scale))
    scaled[negative] = scaled[negative]*-1
    return scaled

def scale_clip(to_scale, clip):
    to_scale = np.clip(to_scale, -clip, clip)
    negative = to_scale < 0
    scaled = np.sqrt(np.abs(to_scale))
    scaled[negative] = scaled[negative]*-1
    return scaled

def log_sign(to_scale):
    negative = to_scale < 0
    scaled = np.log(np.abs(to_scale))
    scaled[negative] = scaled[negative]*-1
    return scaled

gc = scale_clip(gc, 4096)
mc = scale_clip(mc, 256)

In [ ]:
pcs = scale_clip(pc, 256)

In [ ]:
pc_log = log_sign(pc)

In [ ]:
gc_range = (-64,64)
mc_range = (-16,16)

plt.figure(figsize=(10,7))
plt.subplot(2,2,1)
plt.hist(gc, bins=100, range=gc_range)
plt.title('Gaussian Curvature')
plt.subplot(2,2,2)
plt.hist(mc, bins=100, range=mc_range)
plt.title("Mean Curvature")

ax = plt.subplot(2,2,3,projection='3d')
graph = lm_models['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=gc_range[0],
    vmax=gc_range[1],
    cmap='seismic',
    c=gc
)
ax.set_box_aspect([1,1,1])
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])

ax2 = plt.subplot(2,2,4,projection='3d')
graph = lm_models['pretrained'][0][object_n]
model_pos = graph.pos
s = ax2.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=mc_range[0],
    vmax=mc_range[1],
    cmap='seismic',
    c=mc
)
ax2.set_box_aspect([1,1,1])
ax2.set_xticks([]), ax2.set_yticks([]), ax2.set_zticks([])
plt.show()

print(f"gc in range ({max(gc)}, {min(gc)})")
print(f"mc in range ({max(mc)}, {min(mc)})")

In [ ]:
plt.figure()
plt.hist(pc)
plt.show()

In [ ]:
gc_range = (-256,256)
mc_range = (-16,16)

plt.figure(figsize=(10,7))
plt.subplot(2,2,1)
plt.hist(pc, bins=100, range=gc_range)
plt.title('Principal Curvature')
plt.subplot(2,2,2)
plt.hist(pcs, bins=100, range=mc_range)
plt.title("Principal Curvature Scaled")

ax = plt.subplot(2,2,3,projection='3d')
graph = lm_models['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=gc_range[0],
    vmax=gc_range[1],
    cmap='seismic',
    c=pc[:,0]
)
ax.set_box_aspect([1,1,1])
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])

ax2 = plt.subplot(2,2,4,projection='3d')
graph = lm_models['pretrained'][0][object_n]
model_pos = graph.pos
s = ax2.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=mc_range[0],
    vmax=mc_range[1],
    cmap='seismic',
    c=pcs[:,0]
)
ax2.set_box_aspect([1,1,1])
ax2.set_xticks([]), ax2.set_yticks([]), ax2.set_zticks([])
plt.show()

print(f"gc in range ({max(gc)}, {min(gc)})")
print(f"mc in range ({max(mc)}, {min(mc)})")

In [ ]:
gc_range = (-256,256)
mc_range = (-16,16)

plt.figure(figsize=(10,7))
plt.subplot(2,2,1)
plt.hist(pc, bins=100, range=gc_range)
plt.title('Principal Curvature')
plt.subplot(2,2,2)
plt.hist(pc_log, bins=100, range=mc_range)
plt.title("Principal Curvature Logged")

ax = plt.subplot(2,2,3,projection='3d')
graph = lm_models['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=mc_range[0],
    vmax=mc_range[1],
    cmap='seismic',
    c=pc[:,0]
)
ax.set_box_aspect([1,1,1])
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])

ax2 = plt.subplot(2,2,4,projection='3d')
graph = lm_models['pretrained'][0][object_n]
model_pos = graph.pos
s = ax2.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=mc_range[0],
    vmax=mc_range[1],
    cmap='seismic',
    c=pc_log[:,0]
)
ax2.set_box_aspect([1,1,1])
ax2.set_xticks([]), ax2.set_yticks([]), ax2.set_zticks([])
plt.show()

## Plot Features Stored in Models
### Curvature

In [ ]:
object_name = 'mug'
lm_models = lm_models_sensor

fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
graph = lm_models['pretrained'][0][object_name]
model_pos = graph.pos
pc_ids = lm_models['pretrained'][0][object_name].feature_mapping['gaussian_curvature_sc']
# pc_ids = lm_models['pretrained'][0][object_name].feature_mapping['principal_curvatures']
curvs = graph.x[:,pc_ids[0]:pc_ids[1]]
colors = curvs[:,0]
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=-64,
    vmax=64,
    cmap='seismic',
    c=colors
)
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax_range = 0.09
ax.set_xlim(0-ax_range, 0+ax_range)
# ax.set_ylim(1.5-ax_range,1.5+ax_range)
ax.set_ylim(0-ax_range,0+ax_range)
ax.set_zlim(0-ax_range, 0+ax_range)
fig.colorbar(s)
plt.title('Scaled Gaussian Curvature')
# plt.title('Principal Curvature 1')
plt.show()

In [ ]:
object_name = 'mug'
lm_models = lm_models_sensor

graph = lm_models['pretrained'][0][object_name]
model_pos = graph.pos
fig = plt.figure(figsize=(10,7))

ax = plt.subplot(2,2,1,projection='3d')
feat_ids = lm_models['pretrained'][0][object_name].feature_mapping['gaussian_curvature']
colors = graph.x[:,feat_ids[0]:feat_ids[1]]
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=-1000,
    vmax=1000,
    cmap='seismic',
    c=colors
)
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title('Gaussian Curvature')
plt.colorbar(s)

ax = plt.subplot(2,2,2,projection='3d')
feat_ids = lm_models['pretrained'][0][object_name].feature_mapping['mean_curvature']
colors = graph.x[:,feat_ids[0]:feat_ids[1]]
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=-200,
    vmax=200,
    cmap='seismic',
    c=colors
)
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title('Mean Curvature')
plt.colorbar(s)

ax = plt.subplot(2,2,3,projection='3d')
feat_ids = lm_models['pretrained'][0][object_name].feature_mapping['gaussian_curvature_sc']
colors = graph.x[:,feat_ids[0]:feat_ids[1]]
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=-64,
    vmax=64,
    cmap='seismic',
    c=colors
)
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title('Scaled Gaussian Curvature')
plt.colorbar(s)

ax = plt.subplot(2,2,4,projection='3d')
feat_ids = lm_models['pretrained'][0][object_name].feature_mapping['mean_curvature_sc']
colors = graph.x[:,feat_ids[0]:feat_ids[1]]
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=-16,
    vmax=16,
    cmap='seismic',
    c=colors
)
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title('Scaled Mean Curvature')
plt.colorbar(s)

plt.suptitle('From Sensor', fontsize=15)
plt.show()

In [ ]:
object_name = 'potted_meat_can'

graph_sensor = lm_models_sensor['pretrained'][object_name]
model_pos_sensor = graph_sensor.pos
graph_mesh = lm_models_mesh['pretrained'][object_name]
model_pos_mesh = graph_mesh.pos
fig = plt.figure(figsize=(10,7))

ax = plt.subplot(2,2,1,projection='3d')
feat_ids = lm_models_sensor['pretrained'][object_name].feature_mapping['gaussian_curvature_sc']
colors = graph_sensor.x[:,feat_ids[0]:feat_ids[1]]
s = ax.scatter(
    model_pos_sensor[:, 0],
    model_pos_sensor[:, 1],
    model_pos_sensor[:, 2],
    s=5,
    vmin=-64,
    vmax=64,
    cmap='seismic',
    c=colors
)
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title('Scaled Gaussian Curvature - Sensor')
plt.colorbar(s)

ax = plt.subplot(2,2,2,projection='3d')
feat_ids = lm_models_sensor['pretrained'][object_name].feature_mapping['mean_curvature_sc']
colors = graph_sensor.x[:,feat_ids[0]:feat_ids[1]]
s = ax.scatter(
    model_pos_sensor[:, 0],
    model_pos_sensor[:, 1],
    model_pos_sensor[:, 2],
    s=5,
    vmin=-16,
    vmax=16,
    cmap='seismic',
    c=colors
)
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title('Scaled Mean Curvature - Sensor')
plt.colorbar(s)

ax = plt.subplot(2,2,3,projection='3d')
feat_ids = lm_models_mesh['pretrained'][object_name].feature_mapping['gaussian_curvature_sc']
colors = graph_mesh.x[:,feat_ids[0]:feat_ids[1]]
s = ax.scatter(
    model_pos_mesh[:, 0],
    model_pos_mesh[:, 1],
    model_pos_mesh[:, 2],
    s=5,
    vmin=-64,
    vmax=64,
    cmap='seismic',
    c=colors
)
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title('Scaled Gaussian Curvature - Mesh')
plt.colorbar(s)

ax = plt.subplot(2,2,4,projection='3d')
feat_ids = lm_models_mesh['pretrained'][object_name].feature_mapping['mean_curvature_sc']
colors = graph_mesh.x[:,feat_ids[0]:feat_ids[1]]
s = ax.scatter(
    model_pos_mesh[:, 0],
    model_pos_mesh[:, 1],
    model_pos_mesh[:, 2],
    s=5,
    vmin=-16,
    vmax=16,
    cmap='seismic',
    c=colors
)
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_title('Scaled Mean Curvature - Mesh')
plt.colorbar(s)

plt.suptitle('Comparison Between Sensor and Mesh', fontsize=15)
plt.show()

In [ ]:
plot_ids_mesh = graph_mesh.feature_mapping['principal_curvatures']
plot_ids_sensor = graph_sensor.feature_mapping['principal_curvatures']

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
range = (-50,50)
plt.hist(np.array(graph_sensor.x[:,plot_ids_sensor[0]]), alpha=0.6, bins=100, range=range, color='darkblue', density=True)
plt.hist(np.array(graph_sensor.x[:,plot_ids_sensor[0]+1]), alpha=0.6, range=range, bins=100, color='lightblue', density=True)
plt.hist(np.array(graph_mesh.x[:,plot_ids_mesh[0]]), alpha=0.6, bins=100, range=range, color="darkgreen", density=True)
plt.hist(np.array(graph_mesh.x[:,plot_ids_mesh[0] + 1]), alpha=0.6, bins=100, range=range, color="lightgreen", density=True)
plt.legend(["sensor - k1", "sensor - k2", "mesh - k1", "mesh - k2"])


plt.subplot(1,2,2)
plot_ids_mesh_g = graph_mesh.feature_mapping['gaussian_curvature_sc'][0]
plot_ids_sensor_g = graph_sensor.feature_mapping['gaussian_curvature_sc'][0]
range = (-64,64)
plt.hist(np.array(graph_sensor.x[:,plot_ids_sensor_g]), alpha=0.6, bins=100, range=range, color='darkblue', density=True)
plt.hist(np.array(graph_mesh.x[:,plot_ids_mesh_g]), alpha=0.6, bins=100, range=range, color="darkgreen", density=True)
plt.legend(["sensor - gaussian c", "mesh - gaussian c"])#, "mesh - k1", "mesh - k2"])
plt.show()

In [ ]:
color_id_mesh = graph_mesh.feature_mapping['gaussian_curvature_sc'][0]
color_id_sensor = graph_sensor.feature_mapping['gaussian_curvature_sc'][0]
norm_len = 0.02

fig = plt.figure()
# vmin = -64
# vmax = 64
ax = plt.subplot(1,1,1,projection='3d')
# s = ax.scatter(
#     graph_mesh.pos[:, 0],
#     graph_mesh.pos[:, 1],
#     graph_mesh.pos[:, 2],
#     s=5,
#     alpha=0.5,
#     c='green'#np.arctan(np.array(mesh_graph.x[:,color_id_mesh])/100)
# )

# ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
# ax_range = 0.07
# ax.set_xlim(0-ax_range, 0+ax_range)
# ax.set_ylim(0.05-ax_range,0.05+ax_range)
# ax.set_zlim(0-ax_range, 0+ax_range)
# ax.set_title("Learned Locations and Point Normals")

align_offsets = [0.135, -1.475, 0.04]
s2 = ax.scatter(
    graph_sensor.pos[:, 0]+align_offsets[0],
    graph_sensor.pos[:, 1]+align_offsets[1],
    graph_sensor.pos[:, 2]+align_offsets[2],
    s=5,
    alpha=0.5,
    c='blue'#np.arctan(np.array(sensor_graph.x[:,color_id_sensor])/100)
)

# s2 = ax.scatter(
#     graph_mesh.pos[10:12, 0],
#     graph_mesh.pos[10:12, 1],
#     graph_mesh.pos[10:12, 2],
#     s=10,
#     alpha=1,
#     c='red'#np.arctan(np.array(sensor_graph.x[:,color_id_sensor])/100)
# )

# for p_id, p in enumerate(np.array(graph_mesh.pos)):
#     norm = graph_mesh.norm[p_id]
#     ax.plot([p[0], p[0] + norm[0] * norm_len],
#             [p[1], p[1] + norm[1] * norm_len],
#             [p[2], p[2] + norm[2] * norm_len],
#             c='lightgreen',
#            )
    
for p_id, p in enumerate(np.array(graph_sensor.pos)):
    p = p + align_offsets
    norm = graph_sensor.norm[p_id]
    ax.plot([p[0], p[0] + norm[0] * norm_len],
            [p[1], p[1] + norm[1] * norm_len],
            [p[2], p[2] + norm[2] * norm_len],
            c='lightblue',
           )
    
# plt.legend(['From Mesh','From Sensor'])
plt.show()

In [ ]:
node_distances = np.linalg.norm(
                        graph_mesh.pos - graph_mesh.pos[1000],
                        axis=1,
                    )

In [ ]:
min(node_distances)

### HSV

In [ ]:
object_name = 'mug'
lm_models = lm_models_sensor

fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
graph = lm_models['pretrained'][0][object_name]
model_pos = graph.pos
hsv_ids = lm_models['pretrained'][0][object_name].feature_mapping['hsv']
hue = graph.x[:,hsv_ids[0]:hsv_ids[1]]
colors = hue[:,0]
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=0,
    vmax=1,
    cmap='hsv',
    c=colors
)
norm_len = 0.01
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
fig.colorbar(s)
plt.show()

In [ ]:
object_n = "mug"
norm_len=0.01
plt.figure(figsize=(10,10))
ax = plt.subplot(1,1,1,projection='3d')
graph = lm_models_sensor['pretrained'][0][object_n]
model_pos = graph.pos
hsv_ids = lm_models['pretrained'][0][object_name].feature_mapping['hsv']
hue = graph.x[:,hsv_ids[0]:hsv_ids[1]]
colors = hue[:,0]
curv_dir_ids = graph.feature_mapping['curvature_directions']

s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    cmap='hsv',
    c=colors
)
# for p_id, p in enumerate(np.array(model_pos)):
#     norm = graph.norm[p_id]
#     ax.plot([p[0], p[0] + norm[0] * norm_len],
#             [p[1], p[1] + norm[1] * norm_len],
#             [p[2], p[2] + norm[2] * norm_len],
#             c='royalblue',
#            )
#     cd1 = graph.x[p_id][curv_dir_ids[0]:curv_dir_ids[0]+3]
#     cd2 = graph.x[p_id][curv_dir_ids[0]+3:curv_dir_ids[0]+6]
#     ax.plot([p[0], p[0] + cd1[0] * norm_len],
#             [p[1], p[1] + cd1[1] * norm_len],
#             [p[2], p[2] + cd1[2] * norm_len],
#             c='red',
#            )
#     ax.plot([p[0], p[0] + cd2[0] * norm_len * 0.5],
#             [p[1], p[1] + cd2[1] * norm_len * 0.5],
#             [p[2], p[2] + cd2[2] * norm_len * 0.5],
#             c='orange',
#            )
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.show()

### Locations (general plotting function)

In [ ]:
mug_graph = lm_models['pretrained'][0]['mug']
bowl_graph = lm_models['pretrained'][0]['bowl']

In [ ]:
plot_graph(mug_graph, show_edges=False, rotation=20)
plt.show()

# Feature Change Models

In [ ]:
log_path = os.path.expanduser("~/tbp/tbp.monty/projects/monty_runs/logs/")
placeholder_exp_path = log_path + "feature_pred_tests/"

pretrain_path = os.path.expanduser("~/tbp/data/pretrained_models/")
# sensor_pretrained_dict_fc = pretrain_path + "ycb_high_res/supervised_pre_training/pretrained/"


sensor_pretrained_dict_fc = pretrain_path + "ycb_feature_change_S1_low_tol/supervised_pre_training_feature_change/pretrained/"
sensor_pretrained_dict_fc = pretrain_path + "ycb_spiral_policy_s5/supervised_pre_training/pretrained"
_, _, _, lm_models_sensor_fc = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=sensor_pretrained_dict_fc,
                               )

In [ ]:
object_name = 'master_chef_can'
lm_models = lm_models_sensor_fc

fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
graph = lm_models['pretrained'][0][object_name]
model_pos = graph.pos
hsv_ids = lm_models['pretrained'][0][object_name].feature_mapping['hsv']
hue = graph.x[:,hsv_ids[0]:hsv_ids[0]+1]
colors = hue[:,0]
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=0,
    vmax=1,
    cmap='hsv',
    c=colors
)
norm_len = 0.01
ax.set_box_aspect([1,1,1])
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
fig.colorbar(s)
plt.show()

In [ ]:
object_name = 'mug'
lm_models = lm_models_sensor_fc

fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
graph = lm_models['pretrained'][0][object_name]
model_pos = graph.pos
# hsv_ids = lm_models['pretrained'][0][object_name].feature_mapping['hsv']
curv_ids = lm_models['pretrained'][0][object_name].feature_mapping['principal_curvatures_log']
curv = graph.x[:,curv_ids[0]:curv_ids[1]]
colors = curv[:,0]
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=5,
    vmin=-10,
    vmax=10,
    cmap='seismic',
    c=colors
)
norm_len = 0.01
ax.set_box_aspect([1,1,0.7])
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
fig.colorbar(s)
plt.show()

In [ ]:
model_pos.shape

In [ ]:
plt.figure()
plt.hist(np.array(curv))
plt.show()

## Augmented Sparse Models

In [ ]:
# placeholder_exp_path = os.path.expanduser("~/tbp/tbp.monty/projects/monty_runs/evidence_tests/")
placeholder_exp_path = os.path.expanduser("~/tbp/results/monty/projects/monty_runs/evidence_tests/")
save_path = os.path.expanduser("~/tbp/results/monty/figures/models/")

pretrain_path = os.path.expanduser("~/tbp/results/monty/pretrained_models/")

pretrained_dict_0001 = pretrain_path + "pretrained_ycb/supervised_pre_training_10obj_default_base/pretrained/"
_, _, _, lm_models_0001 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_0001,
                               )

pretrained_dict_001 = pretrain_path + "pretrained_ycb/supervised_pre_training_10obj_001dist_base/pretrained/"
_, _, _, lm_models_001 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_001,
                               )

pretrained_dict_005 = pretrain_path + "pretrained_ycb/supervised_pre_training_10obj_005dist_base/pretrained/"
_, _, _, lm_models_005 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_005,
                               )

pretrained_dict_01 = pretrain_path + "pretrained_ycb/supervised_pre_training_10obj_01dist_base/pretrained/"
_, _, _, lm_models_01 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_01,
                               )

# ------ augmented
pretrained_dict_aug_0001 = pretrain_path + "pretrained_ycb/supervised_additional_training_touch_0001dist_10obj/pretrained/"
_, _, _, lm_models_aug_0001 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_aug_0001,
                               )

pretrained_dict_aug_001 = pretrain_path + "pretrained_ycb/supervised_additional_training_touch_001dist_10obj/pretrained/"
_, _, _, lm_models_aug_001 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_aug_001,
                               )

pretrained_dict_aug_005 = pretrain_path + "pretrained_ycb/supervised_additional_training_touch_005dist_10obj/pretrained/"
_, _, _, lm_models_aug_005 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_aug_005,
                               )

pretrained_dict_aug_01 = pretrain_path + "pretrained_ycb/supervised_additional_training_touch_10obj_01/pretrained/"
_, _, _, lm_models_aug_01 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_aug_01,
                               )

pretrained_dict_aug_01on005 = pretrain_path + "pretrained_ycb/supervised_additional_training_touch_01on005_10obj/pretrained/"
_, _, _, lm_models_aug_01on005 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_aug_01on005,
                               )

pretrained_dict_touch_005 = pretrain_path + "pretrained_ycb/only_touch_training_10obj_005dist/pretrained/"
_, _, _, lm_models_touch005 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_touch_005,
                               )

In [ ]:
lm_models_aug_0001['0'].keys()

In [ ]:
lm_models_0001['pretrained'][0].keys()

In [ ]:
object_n = 'spoon'

plt.figure(figsize=(15,12))
ax = plt.subplot(3,4,1,projection='3d')
graph = lm_models_0001['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
)
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.title(f"mmd=0.0001\nnum_points = {graph.x.shape[0]}")

ax = plt.subplot(3,4,2,projection='3d')
graph = lm_models_001['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
)
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.title(f"mmd=0.001\nnum_points = {graph.x.shape[0]}")

ax = plt.subplot(3,4,3,projection='3d')
graph = lm_models_005['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
)
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.title(f"mmd=0.005\nnum_points = {graph.x.shape[0]}")

ax = plt.subplot(3,4,4,projection='3d')
graph = lm_models_01['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
)
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.title(f"mmd=0.01\nnum_points = {graph.x.shape[0]}")

# -------

ax = plt.subplot(3,4,5,projection='3d')
graph = lm_models_aug_0001['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
)
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.title(f"mmd=0.0001\nnum_points = {graph.x.shape[0]}")

ax = plt.subplot(3,4,6,projection='3d')
graph = lm_models_aug_001['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
)
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.title(f"models learned with vision + touch\n\n"
    f"mmd=0.001\nnum_points = {graph.x.shape[0]}")

ax = plt.subplot(3,4,7,projection='3d')
graph = lm_models_aug_005['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
)
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.title(f"mmd=0.005\nnum_points = {graph.x.shape[0]}")

ax = plt.subplot(3,4,8,projection='3d')
graph = lm_models_aug_01['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
)
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.title(f"mmd=0.01\nnum_points = {graph.x.shape[0]}")

#--------------------

ax = plt.subplot(3,4,11,projection='3d')
graph = lm_models_touch005['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
)
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.title(f"mmd=0.005 just touch\nnum_points = {graph.x.shape[0]}")

ax = plt.subplot(3,4,12,projection='3d')
graph = lm_models_aug_01on005['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
)
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.title(f"mmd=0.005 vision, 0.01 touch\nnum_points = {graph.x.shape[0]}")


plt.suptitle("Models learned with vision policy")
# plt.show()
plt.savefig(
            save_path + f"augmented_models_more2_{object_n}.png",
            bbox_inches="tight",
            dpi=300,
        )

In [ ]:
plot_graph(lm_models_touch005['pretrained'][0]['spoon'])
plt.show()

In [ ]:
object_n = "bowl"
norm_len=0.005
plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
graph = lm_models_005['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
)
for p_id, p in enumerate(np.array(model_pos)):
    norm = graph.norm[p_id]
    ax.plot([p[0], p[0] + norm[0] * norm_len],
            [p[1], p[1] + norm[1] * norm_len],
            [p[2], p[2] + norm[2] * norm_len],
            c='blue',
           )
    
graph = lm_models_touch005['pretrained'][0][object_n]
model_pos = graph.pos
s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=1,
    color='orange'
)
for p_id, p in enumerate(np.array(model_pos)):
    norm = graph.norm[p_id]
    ax.plot([p[0], p[0] + norm[0] * norm_len],
            [p[1], p[1] + norm[1] * norm_len],
            [p[2], p[2] + norm[2] * norm_len],
            c='orange',
           )
ax.set_aspect('equal')
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.view_init(-45, 90)
plt.show()

In [ ]:
pretrained_dict_touch_10u_005 = pretrain_path + "pretrained_ycb/supervised_additional_training_10distinctobj_01on005/pretrained/"
_, _, _, lm_models_touch_10u_005 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_touch_10u_005,
                               )

In [ ]:
lm_models_touch_10u_005['pretrained'][0].keys()

In [ ]:
pretrained_dict_touch_10u_005 = pretrain_path + "pretrained_ycb_v3/vision_1lm_10distinctobj/pretrained/"
_, _, _, lm_models_touch_10u_005 = load_stats(placeholder_exp_path,
                                load_train=False,
                                load_eval=False,
                                load_detailed=False,
                                pretrained_dict=pretrained_dict_touch_10u_005,
                               )

In [ ]:
lm_models_touch_10u_005['pretrained'][0].keys()

In [ ]:
plt.figure(figsize=(15,7))
for i, obj in enumerate(list(lm_models_touch_10u_005['pretrained'][0].keys())):
    ax = plt.subplot(2,5,i+1,projection='3d')
    graph = lm_models_touch_10u_005['pretrained'][0][obj]
    model_pos = graph.pos
    s = ax.scatter(
        model_pos[:, 0],
        model_pos[:, 1],
        model_pos[:, 2],
        s=1,
    )
    ax.set_aspect('equal')
    ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
    ax.view_init(-45, 90)
    plt.title(f"{obj}\nnum_points = {graph.x.shape[0]}")
plt.suptitle("Models Learned with Vision (mmd=0.001, lower feat ths)")
plt.savefig(
            save_path + f"current_models_vision_distinct.png",
            bbox_inches="tight",
            dpi=300,
        )